In [2]:
import csv
import sys

In [65]:
DEBUG         = False
line_count    = 0
mth_cnt       = 0
total_P_and_L = 0.0
total_change  = 0.0
prev_pl_amt   = 0.0
max_chg       = 0.0
max_month     = ""
min_chg       = 0.0
min_month     = ""
mth, pl_amt,change = "", 0.0, 0.0

def Save_2_File(data=None, file_name="./data/FinSummary.txt"):
    """ Writes the data to the specified file """
    
    if data is None:
        return False,None
    else:
        try:
            f = open(file_name,"wt")
            f.write(data)
            f.close()
        except OSError as err:
            return False, err.strerror
        
        return True, file_name

def OutputSummary(mth_cnt, total_P_and_L,total_change,max_month,max_chg,min_month,min_chg):
    """
        Generates a string with the summary of the output data
    """
    result = "Financial Analysis\n"
    result += ("-" * 28) + '\n'
    result += (f'Total Months: {mth_cnt}\n')
    result += (f'Total:\t\t{total_P_and_L:16,.2f}\n')
    result += (f'Total Change: {total_change:18,.2f}\n')
    result += (f'Average Change:{total_change/(mth_cnt-1):17,.2f}\n')
    result += (f'Greatest Increase in Profits: {max_month} (${max_chg:15,.2f})\n')
    result += (f'Greatest Decrease in Profits: {min_month} (${min_chg:15,.2f})\n')

    return result
    
with open('data/budget_data.csv') as csv_budget:
    budget_reader = csv.reader(csv_budget, delimiter=',')
    
    for row in budget_reader:
        if line_count == 0:
            if DEBUG:
                print(f'Column names are {", ".join(row)}')
        else:
            mth, pl_amt    = row[0], float(row[1])
            total_P_and_L += pl_amt
            
            # Skip the first 'prev_pl_amt'
            if line_count > 1:
                change         = (pl_amt - prev_pl_amt)
                total_change  += change

            if DEBUG:
                print(f'{line_count:2} Month:\t{mth}  amount: {pl_amt:15,.2f} \tchange: {change:15,.2f} \ttotal Change:{total_change:15,.2f}')
            
            if change > 0:
                if change > max_chg:
                    max_chg = change
                    max_month = mth
            else:
                if change < min_chg:
                    min_chg = change
                    min_month = mth
            
            mth_cnt    += 1
            prev_pl_amt = pl_amt
        
        total_change += (pl_amt - prev_pl_amt)
        line_count   += 1
    
summary_data = OutputSummary(mth_cnt, total_P_and_L,total_change,max_month,max_chg,min_month,min_chg)
success, fname = Save_2_File(summary_data)
saved_state = f'Wrote data to {fname}' if success else f'Sorry not able to write to file beacuse {fname}'

print(summary_data)
print(saved_state)

Financial Analysis
----------------------------
Total Months: 86
Total:		   38,382,578.00
Total Change:        -196,785.00
Average Change:        -2,315.12
Greatest Increase in Profits: Feb-2012 ($   1,926,159.00)
Greatest Decrease in Profits: Sep-2013 ($  -2,196,167.00)

Wrote data to ./data/FinSummary.txt


In [16]:
# Let's try that with Pandas

In [17]:
import pandas as pd

In [19]:
df = pd.read_csv("./data/budget_data.csv")

In [22]:
total_PL = df['Profit/Losses'].sum()

In [41]:
df['prev mth PL'] = df['Profit/Losses'].shift(1) #,fill_value=0)

In [42]:
df['change'] = df['Profit/Losses'] - df['prev mth PL']

In [61]:
f"Number of months: {df['Date'].count():14}",f"The Average Change: {df['change'].mean():12,.2f}", f"The minimum change {df['change'].min():13,.2f}", f"The minimum change {df['change'].max():13,.2f}",f"The count of change {df['change'].count():12}"

('Number of months:             86',
 'The Average Change:    -2,315.12',
 'The minimum change -2,196,167.00',
 'The minimum change  1,926,159.00',
 'The count of change           85')

In [44]:
df

,Date,Profit/Losses,prev mth PL,change
0,Jan-2010,867884,NaN,NaN
1,Feb-2010,984655,867884.0,116771.0
2,Mar-2010,322013,984655.0,-662642.0
3,Apr-2010,-69417,322013.0,-391430.0
4,May-2010,310503,-69417.0,379920.0
...,...,...,...,...
81,Oct-2016,102685,768450.0,-665765.0
82,Nov-2016,795914,102685.0,693229.0
83,Dec-2016,60988,795914.0,-734926.0
84,Jan-2017,138230,60988.0,77242.0
